In [2]:
import pandas as pd
import geopandas as gpd
import folium

## Get 2016 personal income data for Toronto area by census tract
1. Download Census tract data https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page_Download-Telecharger.cfm?Lang=E&Tab=1&Geo1=CSD&Code1=3520005&Geo2=PR&Code2=35&Data=Count&SearchText=Toronto&SearchType=Begins&SearchPR=01&B1=All&TABID=1
1. Unzip and save it in the same folder as this notebook (about 1.3Gb unzipped)
1. Run the following code to generate two files: income, and income as percent of Toronto (both files are by census tract)
1. Census tracts are filtered to Toronto area (those starting with 535)

We select `Median total income in 2015 among recipients ($)` variable.

**To calculate %, divide it by Toronto Census metro area value of 31,705** (taken from https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=CMACA&Code1=535&Geo2=PR&Code2=01&SearchText=toronto&SearchType=Begins&SearchPR=01&B1=All&TABID=1&type=0)

In [40]:
census2016 = pd.read_csv('./98-401-X2016043_English_CSV_data.csv', dtype={1:str, 3: str, 11: str})

In [41]:
# Extract `Median total income in 2015 among recipients ($)` variable
census2016_income = census2016[
        (census2016['DIM: Profile of Census Tracts (2247)'] == 'Median total income in 2015 among recipients ($)')
        & (census2016['GEO_CODE (POR)'].str.startswith('535'))
    ].filter(['GEO_CODE (POR)', 'Dim: Sex (3): Member ID: [1]: Total - Sex'])

# Rename columns
census2016_income.columns = ['Tract', 'Income']
census2016_income = census2016_income.set_index('Tract')

# Remove 'x' in income
census2016_income.Income = pd.to_numeric(census2016_income.Income, errors='coerce')

# Save (small file size, acceptable for GitHub)
census2016_income.to_csv('census/census2016-income.csv', index=True)

# Save income as percent into a separate file
round(census2016_income / 31705 * 100, 1).to_csv('census/census2016-income-as-percent.csv', index=True)

## Create a Leaflet map with `folium`

In [69]:
# Initialize map
m = folium.Map(
    location=[43.6529, -79.3849],
    tiles=None,
    zoom_start=12,
    control_scale=True,
)

# Add CartoDB Positron baselayer
tiles = folium.TileLayer(
    tiles='CartoDB positron',
    control=False,
    zoom_start=12,
)

# Add link to GitHub repo to attribution
tiles.options['attribution'] = '{} | {}'.format(
    '<a href="https://github.com/metrohistory/toronto-cma-income-areas">View sources and code</a>',
    tiles.options['attribution']
)

tiles.add_to(m)

### Create Choropleth layer

In [70]:
# Read income data file for choropleth
census2016_income_percent = pd.Series( pd.read_csv('census/census2016-income-as-percent.csv', dtype={0: str}).set_index('Tract').Income )

# Read census tracts GeoJSON
tracts = gpd.read_file('geodata/toronto-cma-tracts-2016-statcan-simplified.geojson').filter(['CTUID', 'geometry'])

# Attach Income column to tracts in order to be able to display it in a tooltip
tracts['Income'] = tracts.CTUID.apply(lambda x: census2016_income_percent.loc[x])

In [71]:
income_choropleth = folium.Choropleth(
    geo_data=tracts,
    name='Income 2015 for Toronto CMA',
    bins=[45, 60, 75, 90, 110, 150, 200, 270],
    data=census2016_income_percent,
    key_on='feature.properties.CTUID',
    fill_color='RdBu',
    line_weight=1,
    line_color='white',
    nan_fill_color='silver',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Median Personal Income as % of Toronto CMA, Census 2016",
    highlight=True
)

# Add Tooltip data
folium.GeoJsonTooltip(
    fields=['CTUID', 'Income'],
    aliases=['Census Tract', 'Income as %']
).add_to(income_choropleth.geojson)

### Add all overlays in order, including Toronto, neighbourhoods, and chotopleth

In [72]:
# Toronto pre-1966
folium.GeoJson(
    f'geodata/toronto-city-pre1966-osm-edit.geojson',
    name='Old City of Toronto (dissolved)',
    control=True,
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': '4',
        'dashArray': '8',
        'interactive': False
    }
).add_to(m)


# Toronto 1998-present
folium.GeoJson(
    f'geodata/toronto-boundary-2016-osm.geojson',
    name='Toronto 1998-present',
    control=True,
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': '4',
        'interactive': False
    }
).add_to(m)


# Etobicoke
folium.GeoJson(
    f'geodata/etobicoke-boundary-osm.geojson',
    name='Etobicoke (dissolved)',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'orange',
        'weight': '4',
        'dashArray': '8',
        'interactive': False
    }
).add_to(m)


# York
folium.GeoJson(
    f'geodata/york-boundary-osm.geojson',
    name='York (dissolved)',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'magenta',
        'weight': '4',
        'dashArray': '8',
        'interactive': False
    }
).add_to(m)


# Forrest Hill
folium.GeoJson(
    f'geodata/forest-hill-boundary-1961-dli.geojson',
    name='Forest Hill (dissolved)',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': '#6600cc',
        'weight': '4',
        'dashArray': '8',
        'interactive': False
    }
).add_to(m)


# Scarborough
folium.GeoJson(
    f'geodata/scarborough-boundary-osm.geojson',
    name='Scarborough (dissolved)',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'green',
        'weight': '4',
        'dashArray': '8',
        'interactive': False
    }
).add_to(m)


# Choropleth is not a layer but a FeatureGroup, so add thin black boundary to it

# Toronto CMA
folium.GeoJson(
    f'geodata/toronto-cma-boundary-2016-dli.geojson',
    name='Toronto CMA',
    control=False,
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': '1',
        'interactive': False
    }
).add_to(income_choropleth)


# Add choropleth to the map
income_choropleth.add_to(m)

### Final touches and save!

In [73]:
# Add layer control
folium.LayerControl(collapsed=False, position='bottomright').add_to(m)

# And all done!
m.save('index.html')

## Manual additions to the generated HTML map

### 1. Add lines to the legend with neighbourhoods boundaries
```html
<style>
  .line {
    display: inline-block;
    vertical-align: middle;
    width: 35px;
  }
</style>

<span class='line' style='border-bottom: 4px dashed orange'></span>
```

### 2. Keep choropleth underneath neighbourhood boundaries always
```js
choropleth_8926c1b6196e461e85cde667cbe9d1dd.on('add', function() {
  this.bringToBack();
}); 
```

### 3. Add white background to legend for readability
```html
<style>
    .legend.leaflet-control {
        background-color: white;
    }
</style>
```

### 4. Add North indicator
```js
var north = L.control({position: 'bottomleft'});
north.onAdd = function(map) {
    var div = L.DomUtil.create('div');
    div.innerHTML = '<img src="north.png" style="height: 30px;" alt="North indicator" title="North indicator">';
    return div;
}
north.addTo(map_9f8605217f1b408b83f4a098798ee48e);
```

### 5. Manually swap text and numbers in choropleth legend
Requires some svg y-axis repositionings here and there.